# KubeFlow Pipeline Using TFX OSS Components

In this notebook, we will demo: 

* Defining a KubeFlow pipeline with Python DSL
* Submiting it to Pipelines System
* Customize a step in the pipeline

We will use a pipeline that includes some TFX OSS components such as [TFDV](https://github.com/tensorflow/data-validation), [TFT](https://github.com/tensorflow/transform), [TFMA](https://github.com/tensorflow/model-analysis).

## Setup

In [ ]:
# Install Pipeline SDK
!pip3 install https://storage.googleapis.com/ml-pipeline/release/0.1.1/kfp.tar.gz --upgrade
    
# Component building requires this package. For now you need to install it explicitly.
!pip3 install kubernetes

In [36]:
import kfp
from kfp import compiler
import kfp.dsl as dsl
import kfp.notebook


# Set your output and project. !!!Must Do before you can proceed!!!
OUTPUT_DIR = 'Your-Gcs-Path' # Such as gs://bucket/objact/path
PROJECT_NAME = 'Your-Gcp-Project-Name'
BASE_IMAGE='gcr.io/%s/pusherbase:dev' % PROJECT_NAME
TARGET_IMAGE='gcr.io/%s/pusher:dev' % PROJECT_NAME

## Create an Experiment in the Pipeline System

Pipeline system requires an "Experiment" to group pipeline runs. You can create a new experiment, or call client.list_experiments() to get existing ones.

In [13]:
# Note that this notebook should be running in JupyterHub in the same cluster as the pipeline system.
# Otherwise it will fail to talk to the pipeline system.
client = kfp.Client()
exp = client.create_experiment(name='demo')

## Test Run a Pipeline

In [29]:
# Download a pipeline package
!gsutil cp gs://ml-pipeline-playground/coin.tar.gz .

Copying gs://ml-pipeline-playground/coin.tar.gz...
/ [1 files][  978.0 B/  978.0 B]                                                
Operation completed over 1 objects/978.0 B.                                      


In [30]:
run = client.run_pipeline(exp.id, 'coin', 'coin.tar.gz')

## Define a Pipeline

Authoring a pipeline is just like authoring a normal Python function. The pipeline function describes the topology of the pipeline. Each step in the pipeline is typically a ContainerOp --- a simple class or function describing how to interact with a docker container image. In the below pipeline, all the container images referenced in the pipeline are already built. The pipeline starts with a TFDV step which is used to infer the schema of the data. Then it uses TFT to transform the data for training. After a single node training step, it analyze the test data predictions and generate a feature slice metrics view using a TFMA component. At last, it deploys the model to TF-Serving inside the same cluster.

In [14]:
import kfp.dsl as dsl


# Below are a list of helper functions to wrap the components to provide a simpler interface for pipeline function.
def dataflow_tf_data_validation_op(inference_data: 'GcsUri', validation_data: 'GcsUri', column_names: 'GcsUri[text/json]', key_columns, project: 'GcpProject', mode, validation_output: 'GcsUri[Directory]', step_name='validation'):
    return dsl.ContainerOp(
        name = step_name,
        image = 'gcr.io/ml-pipeline/ml-pipeline-dataflow-tfdv:dev',
        arguments = [
            '--csv-data-for-inference', inference_data,
            '--csv-data-to-validate', validation_data,
            '--column-names', column_names,
            '--key-columns', key_columns,
            '--project', project,
            '--mode', mode,
            '--output', validation_output,
        ],
        file_outputs = {
            'output': '/output.txt',
            'schema': '/output_schema.json',
        }
    )

def dataflow_tf_transform_op(train_data: 'GcsUri', evaluation_data: 'GcsUri', schema: 'GcsUri[text/json]', project: 'GcpProject', preprocess_mode, preprocess_module: 'GcsUri[text/code/python]', transform_output: 'GcsUri[Directory]', step_name='preprocess'):
    return dsl.ContainerOp(
        name = step_name,
        image = 'gcr.io/ml-pipeline/ml-pipeline-dataflow-tft:0.0.42',
        arguments = [
            '--train', train_data,
            '--eval', evaluation_data,
            '--schema', schema,
            '--project', project,
            '--mode', preprocess_mode,
            '--preprocessing-module', preprocess_module,
            '--output', transform_output,
        ],
        file_outputs = {'transformed': '/output.txt'}
    )


def tf_train_op(transformed_data_dir, schema: 'GcsUri[text/json]', learning_rate: float, hidden_layer_size: int, steps: int, target: str, preprocess_module: 'GcsUri[text/code/python]', training_output: 'GcsUri[Directory]', step_name='training'):
    return dsl.ContainerOp(
        name = step_name,
        image = 'gcr.io/ml-pipeline/ml-pipeline-kubeflow-tf-trainer:0.0.42',
        arguments = [
            '--transformed-data-dir', transformed_data_dir,
            '--schema', schema,
            '--learning-rate', learning_rate,
            '--hidden-layer-size', hidden_layer_size,
            '--steps', steps,
            '--target', target,
            '--preprocessing-module', preprocess_module,
            '--job-dir', training_output,
        ],
        file_outputs = {'train': '/output.txt'}
    )

def dataflow_tf_model_analyze_op(model: 'TensorFlow model', evaluation_data: 'GcsUri', schema: 'GcsUri[text/json]', project: 'GcpProject', analyze_mode, analyze_slice_column, analysis_output: 'GcsUri', step_name='analysis'):
    return dsl.ContainerOp(
        name = step_name,
        image = 'gcr.io/ml-pipeline/ml-pipeline-dataflow-tfma:0.0.42',
        arguments = [
            '--model', model,
            '--eval', evaluation_data,
            '--schema', schema,
            '--project', project,
            '--mode', analyze_mode,
            '--slice-columns', analyze_slice_column,
            '--output', analysis_output,
        ],
        file_outputs = {'analysis': '/output.txt'}
    )


def dataflow_tf_predict_op(evaluation_data: 'GcsUri', schema: 'GcsUri[text/json]', target: str, model: 'TensorFlow model', predict_mode, project: 'GcpProject', prediction_output: 'GcsUri', step_name='prediction'):
    return dsl.ContainerOp(
        name = step_name,
        image = 'gcr.io/ml-pipeline/ml-pipeline-dataflow-tf-predict:0.0.42',
        arguments = [
            '--data', evaluation_data,
            '--schema', schema,
            '--target', target,
            '--model',  model,
            '--mode', predict_mode,
            '--project', project,
            '--output', prediction_output,
        ],
        file_outputs = {'prediction': '/output.txt'}
    )

def kubeflow_deploy_op(model: 'TensorFlow model', tf_server_name, step_name='deploy'):
    return dsl.ContainerOp(
        name = step_name,
        image = 'gcr.io/ml-pipeline/ml-pipeline-kubeflow-deployer:dev',
        arguments = [
            '--model-path', model,
            '--server-name', tf_server_name
        ]
    )


# The pipeline definition
@dsl.pipeline(
  name='TFX Taxi Cab Classification Pipeline Example',
  description='Example pipeline that does classification with model analysis based on a public BigQuery dataset.'
)
def taxi_cab_classification(
    output,
    project,
    column_names=dsl.PipelineParam(name='column-names', value='gs://ml-pipeline-playground/tfx/taxi-cab-classification/column-names.json'),
    key_columns=dsl.PipelineParam(name='key-columns', value='trip_start_timestamp'),
    train=dsl.PipelineParam(name='train', value='gs://ml-pipeline-playground/tfx/taxi-cab-classification/train.csv'),
    evaluation=dsl.PipelineParam(name='evaluation', value='gs://ml-pipeline-playground/tfx/taxi-cab-classification/eval.csv'),
    validation_mode=dsl.PipelineParam(name='validation-mode', value='local'),
    preprocess_mode=dsl.PipelineParam(name='preprocess-mode', value='local'),
    preprocess_module: dsl.PipelineParam=dsl.PipelineParam(name='preprocess-module', value='gs://ml-pipeline-playground/tfx/taxi-cab-classification/preprocessing.py'),
    target=dsl.PipelineParam(name='target', value='tips'),
    learning_rate=dsl.PipelineParam(name='learning-rate', value=0.1),
    hidden_layer_size=dsl.PipelineParam(name='hidden-layer-size', value='1500'),
    steps=dsl.PipelineParam(name='steps', value=3000),
    predict_mode=dsl.PipelineParam(name='predict-mode', value='local'),
    analyze_mode=dsl.PipelineParam(name='analyze-mode', value='local'),
    analyze_slice_column=dsl.PipelineParam(name='analyze-slice-column', value='trip_start_hour')):
    
    validation_output = '%s/{{workflow.name}}/validation' % output
    transform_output = '%s/{{workflow.name}}/transformed' % output
    training_output = '%s/{{workflow.name}}/train' % output
    analysis_output = '%s/{{workflow.name}}/analysis' % output
    prediction_output = '%s/{{workflow.name}}/predict' % output
    tf_server_name = 'taxi-cab-classification-model-{{workflow.name}}'

    validation = dataflow_tf_data_validation_op(train, evaluation, column_names, key_columns, project, validation_mode, validation_output)
    schema = '%s/schema.json' % validation.outputs['output']

    preprocess = dataflow_tf_transform_op(train, evaluation, schema, project, preprocess_mode, preprocess_module, transform_output)
    training = tf_train_op(preprocess.output, schema, learning_rate, hidden_layer_size, steps, target, preprocess_module, training_output)
    analysis = dataflow_tf_model_analyze_op(training.output, evaluation, schema, project, analyze_mode, analyze_slice_column, analysis_output)
    prediction = dataflow_tf_predict_op(evaluation, schema, target, training.output, predict_mode, project, prediction_output)
    deploy = kubeflow_deploy_op(training.output, tf_server_name)

## Submit the run

In [27]:
# Compile it into a tar package.
compiler.Compiler().compile(taxi_cab_classification,  'tfx.tar.gz')

# Submit a run.
run = client.run_pipeline(exp.id, 'tfx', 'tfx.tar.gz',
                          params={'output': OUTPUT_DIR,
                                  'project': PROJECT_NAME})

## Customize a step in the above pipeline

Let's say I got the pipeline source code from github, and I want to modify the pipeline a little bit by swapping the last deployer step with my own deployer. Instead of tf-serving deployer, I want to deploy it to Cloud ML Engine service.

### Create and test a python function for the new deployer

In [ ]:
# in order to run it locally we need a python package
!pip3 install google-api-python-client

In [30]:
@dsl.python_component(
    name='cmle_deployer',
    description='deploys a model to GCP CMLE',
    base_image=BASE_IMAGE
)
def deploy_model(model_dot_version: str, model_path: str, gcp_project: str, runtime: str):

    from googleapiclient import discovery
    from tensorflow.python.lib.io import file_io
    import os
    
    model_path = file_io.get_matching_files(os.path.join(model_path, 'export', 'export', '*'))[0]
    api = discovery.build('ml', 'v1')
    model_name, version_name = model_dot_version.split('.')
    body = {'name': model_name}
    parent = 'projects/%s' % gcp_project
    try:
        api.projects().models().create(body=body, parent=parent).execute()
    except:
        # Trying to create an already existing model gets an error. Ignore it.
        pass

    import time

    body = {
        'name': version_name,
        'deployment_uri': model_path,
        'runtime_version': runtime
    }

    full_mode_name = 'projects/%s/models/%s' % (gcp_project, model_name)
    response = api.projects().models().versions().create(body=body, parent=full_mode_name).execute()
    
    while True:
        response = api.projects().operations().get(name=response['name']).execute()
        if 'done' not in response or response['done'] is not True:
            time.sleep(5)
            print('still deploying...')
        else:
            if 'error' in response:
                print(response['error'])
            else:
                print('Done.')
            break

In [ ]:
# Test the function and make sure it works.
path = 'gs://ml-pipeline-playground/sampledata/taxi/train'
deploy_model('taxidev.beta', path, PROJECT_NAME, '1.9')

### Build a Pipeline Step With the Above Function

Now that we've tested the function locally, we want to build a component that can run as a step in the pipeline. First we need to build a base docker container image. We need TensorFlow and google-api-python-client packages.

In [35]:
%%docker {BASE_IMAGE} {OUTPUT_DIR}
FROM tensorflow/tensorflow:1.10.0-py3
RUN pip3 install google-api-python-client

INFO:root:Checking path: gs://bradley-playground...
INFO:root:Generate build files.
INFO:root:Start a kaniko job for build.
INFO:root:5 seconds: waiting for job to complete
INFO:root:10 seconds: waiting for job to complete
INFO:root:15 seconds: waiting for job to complete
INFO:root:20 seconds: waiting for job to complete
INFO:root:25 seconds: waiting for job to complete
INFO:root:30 seconds: waiting for job to complete
INFO:root:35 seconds: waiting for job to complete
INFO:root:40 seconds: waiting for job to complete
INFO:root:45 seconds: waiting for job to complete
INFO:root:50 seconds: waiting for job to complete
INFO:root:55 seconds: waiting for job to complete
INFO:root:60 seconds: waiting for job to complete
INFO:root:65 seconds: waiting for job to complete
INFO:root:70 seconds: waiting for job to complete
INFO:root:75 seconds: waiting for job to complete
INFO:root:80 seconds: waiting for job to complete
INFO:root:85 seconds: waiting for job to complete
INFO:root:90 seconds: waiti

Once the base docker container image is built, we can build a "target" container image that is base_image plus the python function as entry point. The target container image can be used as a step in a pipeline.

In [20]:
from kfp import compiler

# The return value "DeployerOp" represents a step that can be used directly in a pipeline function
DeployerOp = compiler.build_python_component(
    component_func=deploy_model,
    staging_gcs_path=OUTPUT_DIR,
    target_image=TARET_IMAGE)

INFO:root:Build an image that is based on gcr.io/bradley-playground/pusher:dev and push the image to gcr.io/bradley-playground/pusher:latest
INFO:root:Checking path: gs://bradley-playground...
INFO:root:Generate entrypoint and serialization codes.
INFO:root:Generate build files.
INFO:root:Start a kaniko job for build.
INFO:root:5 seconds: waiting for job to complete
INFO:root:10 seconds: waiting for job to complete
INFO:root:15 seconds: waiting for job to complete
INFO:root:20 seconds: waiting for job to complete
INFO:root:25 seconds: waiting for job to complete
INFO:root:30 seconds: waiting for job to complete
INFO:root:35 seconds: waiting for job to complete
INFO:root:40 seconds: waiting for job to complete
INFO:root:45 seconds: waiting for job to complete
INFO:root:50 seconds: waiting for job to complete
INFO:root:55 seconds: waiting for job to complete
INFO:root:60 seconds: waiting for job to complete
INFO:root:65 seconds: waiting for job to complete
INFO:root:70 seconds: waiting f

### Modify the pipeline with the new deployer

In [24]:
# My New Pipeline. It's almost the same as the original one with the last step deployer replaced.
@dsl.pipeline(
  name='TFX Taxi Cab Classification Pipeline Example',
  description='Example pipeline that does classification with model analysis based on a public BigQuery dataset.'
)
def my_taxi_cab_classification(
    output,
    project,
    model,
    column_names=dsl.PipelineParam(
        name='column-names',
        value='gs://ml-pipeline-playground/tfx/taxi-cab-classification/column-names.json'),
    key_columns=dsl.PipelineParam(name='key-columns', value='trip_start_timestamp'),
    train=dsl.PipelineParam(
        name='train',
        value='gs://ml-pipeline-playground/tfx/taxi-cab-classification/train.csv'),
    evaluation=dsl.PipelineParam(
        name='evaluation',
        value='gs://ml-pipeline-playground/tfx/taxi-cab-classification/eval.csv'),
    validation_mode=dsl.PipelineParam(name='validation-mode', value='local'),
    preprocess_mode=dsl.PipelineParam(name='preprocess-mode', value='local'),
    preprocess_module: dsl.PipelineParam=dsl.PipelineParam(
        name='preprocess-module',
        value='gs://ml-pipeline-playground/tfx/taxi-cab-classification/preprocessing.py'),
    target=dsl.PipelineParam(name='target', value='tips'),
    learning_rate=dsl.PipelineParam(name='learning-rate', value=0.1),
    hidden_layer_size=dsl.PipelineParam(name='hidden-layer-size', value='1500'),
    steps=dsl.PipelineParam(name='steps', value=3000),
    predict_mode=dsl.PipelineParam(name='predict-mode', value='local'),
    analyze_mode=dsl.PipelineParam(name='analyze-mode', value='local'),
    analyze_slice_column=dsl.PipelineParam(name='analyze-slice-column', value='trip_start_hour')):
    
    
    validation_output = '%s/{{workflow.name}}/validation' % output
    transform_output = '%s/{{workflow.name}}/transformed' % output
    training_output = '%s/{{workflow.name}}/train' % output
    analysis_output = '%s/{{workflow.name}}/analysis' % output
    prediction_output = '%s/{{workflow.name}}/predict' % output

    validation = dataflow_tf_data_validation_op(train, evaluation, column_names, key_columns, project, validation_mode, validation_output)
    schema = '%s/schema.json' % validation.outputs['output']

    preprocess = dataflow_tf_transform_op(train, evaluation, schema, project, preprocess_mode, preprocess_module, transform_output)
    training = tf_train_op(preprocess.output, schema, learning_rate, hidden_layer_size, steps, target, preprocess_module, training_output)
    analysis = dataflow_tf_model_analyze_op(training.output, evaluation, schema, project, analyze_mode, analyze_slice_column, analysis_output)
    prediction = dataflow_tf_predict_op(evaluation, schema, target, training.output, predict_mode, project, prediction_output)
    
    # The new deployer. Note that the DeployerOp interface is similar to the function "deploy_model".
    deploy = DeployerOp(gcp_project=project, model_dot_version=model, runtime='1.9', model_path=training.output)

# Submit a new job

In [25]:
compiler.Compiler().compile(my_taxi_cab_classification,  'my-tfx.tar.gz')

run = client.run_pipeline(exp.id, 'my-tfx', 'my-tfx.tar.gz',
                          params={'output': OUTPUT_DIR,
                                  'project': PROJECT_NAME,
                                  'model': 'mytaxi.beta'})